In [1]:
import torch

from model import MSAVAE
from config import create_config
from collections import OrderedDict
from torch.nn.parallel import DistributedDataParallel as DDP
from data import read_msa, greedy_select, pad_msa_sequence

/home/av47/miniconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ckpt = torch.load("weights/model_14.pth")
new_state_dict = OrderedDict()
for k, v in ckpt["model"].items():
    name = k[7:]
    new_state_dict[name] = v

/tmp/ipykernel_974346/2447741691.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("weights/model_14.pth")


In [3]:
config = create_config()
model = MSAVAE(config).to("cuda:0")
model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [5]:
raw_msa = read_msa("databases/data/a3m/1aua_1_B.a3m")
msa = [raw_msa[1:]]
seq = [raw_msa[0]]
single_seq_embeddings, pairwise_seq_embeddings, msa_tokens, mask = pad_msa_sequence(config, seq, msa)

In [7]:
with torch.no_grad():
    pred_msa, perm, mu, logvar = model(single_seq_embeddings, pairwise_seq_embeddings, msa_tokens.unsqueeze(-1).float(), mask)

OutOfMemoryError: CUDA out of memory. Tried to allocate 9.00 GiB. GPU 0 has a total capacity of 79.26 GiB of which 8.01 GiB is free. Process 904267 has 69.38 GiB memory in use. Including non-PyTorch memory, this process has 1.85 GiB memory in use. Of the allocated memory 1.30 GiB is allocated by PyTorch, and 62.97 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)